Build an Excel workbook to simplify manual data extraction (install openpyxl first)

In [ ]:
! pip install openpyxl

Set the path to the documents, random seed, and the number of documents to evaluate.

In [ ]:
import random
from pathlib import Path
import shutil

random.seed(42)
num_to_copy = 50

# randomly select up to 50 files
all_papers = sorted(Path("papers").glob("*.pdf"))

selected_papers = random.sample(all_papers, num_to_copy)
papers = sorted([str(x).split("\\")[-1] for x in selected_papers])

dst_dir = Path("selected_papers")
dst_dir.mkdir(exist_ok=True)
for paper in selected_papers:
    shutil.copy2(paper, dst_dir / paper.name)

print(f"Copied {num_to_copy} papers to {dst_dir}")

Build the Excel workbook to be used for manual data extraction.

In [ ]:
from manual_extraction.utils import build_workbook
import json


questions = json.load(open("questions.json"))
questions = {q["text"]: q["choices"] for q in questions["questions"]}

build_workbook(
    papers,
    questions,
    out_path="manual_extraction/dropdown_menus.xlsm",  # keep .xlsm
    start_row=2,
    max_rows=200,
    template_path="manual_extraction/review_dha_macro.xlsm",  # your VBA template
)

Calculate inter-rater agreement (Cohen's kappa)

In [ ]:
import pandas as pd
import os
import json
import math
from sklearn.metrics import cohen_kappa_score

model_1 = "qwen72b"
model_2 = "qwen7b"

answer_dir_1 = f"answers/{model_1}/"
answer_dir_2 = f"answers/{model_2}/"

# load data
answers_researcher_1 = pd.read_csv(
    "manual_extraction/answers_researcher_1.csv", sep=";"
)
answers_researcher_2 = pd.read_csv(
    "manual_extraction/answers_researcher_2.csv", sep=";"
)


def clean_key(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
        .str.replace("_", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace("-", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.lower()
    )


answers_researcher_1["paper_key"] = clean_key(answers_researcher_1["paper"])
answers_researcher_2["paper_key"] = clean_key(answers_researcher_2["paper"])
answers_researcher_1 = answers_researcher_1.sort_values(by="paper_key")
answers_researcher_2 = answers_researcher_2.sort_values(by="paper_key")

# Global boolean labels collected in identical order
y_llm1, y_llm2, y_researcher_1, y_researcher_2 = [], [], [], []


def norm_list(x):
    if isinstance(x, list):
        xs = x
    else:
        xs = [x]
    return [str(v).replace(" ", "").lower() for v in xs if str(v).strip() != ""]


def get_cell(df, mask, col):
    val = df.loc[mask, col].iloc[0]
    if isinstance(val, float) and math.isnan(val):
        return ""
    return str(val)


for paper_fname in os.listdir(answer_dir_1):
    p1 = os.path.join(answer_dir_1, paper_fname)
    p2 = os.path.join(answer_dir_2, paper_fname)
    if not os.path.exists(p2):
        print(f"WARNING: missing {p2}; skipping file")
        continue

    with open(p1, "r") as f:
        answers_1 = json.load(f)
    with open(p2, "r") as f:
        answers_2 = json.load(f)

    # filename-derived key
    paper_clean_name = paper_fname.replace("_", "").replace(" ", "").lower()
    prefix = paper_clean_name[:35]

    mask_researcher_1 = answers_researcher_1["paper_key"].str[:35].eq(prefix)
    mask_researcher_2 = answers_researcher_2["paper_key"].str[:35].eq(prefix)

    if mask_researcher_1.sum() != 1 or mask_researcher_2.sum() != 1:
        print("WARNING: No unique match found for paper", paper_fname)
        print(f"prefix: {prefix}")
        print(f"researcher 1 paper keys: {answers_researcher_1['paper_key'].str[:35]}")
        print(f"researcher 2 paper keys: {answers_researcher_2['paper_key'].str[:35]}")
        raise Exception(f"No unique match found for paper {paper_fname}")

    # Build a quick dict for LLM2 answers by question for robustness
    llm2_by_q = {aq["question"]: aq for aq in answers_2["answers"]}

    for q1 in answers_1["answers"]:
        qname = q1["question"]
        if qname not in llm2_by_q:
            print(
                f"WARNING: question '{qname}' not in LLM2 for {paper_fname}; skipping"
            )
            continue

        # Normalize LLM answers
        llm_answer_1 = norm_list(q1["answer"])
        llm_answer_2 = norm_list(llm2_by_q[qname]["answer"])

        # Fetch human answers and normalize to sets of ids
        researcher_1_cell = get_cell(answers_researcher_1, mask_researcher_1, qname)
        researcher_2_cell = get_cell(answers_researcher_2, mask_researcher_2, qname)
        researcher_1_ans = [
            s for s in researcher_1_cell.replace(" ", "").lower().split(";") if s
        ]
        researcher_2_ans = [
            s for s in researcher_2_cell.replace(" ", "").lower().split(";") if s
        ]

        # Choices/space of labels — prefer ids from LLM1; sanity-check LLM2's choices if present
        choices = q1.get("choices_ids", [])
        # If LLM2 has a choices list and it's different, fall back to union to stay aligned
        choices2 = llm2_by_q[qname].get("choices_ids", [])
        if choices2 and set(map(str, choices2)) != set(map(str, choices)):
            # robust union (normalized)
            c1 = [str(c).replace(" ", "").lower() for c in choices]
            c2 = [str(c).replace(" ", "").lower() for c in choices2]
            choices = sorted(set(c1).union(c2))
        else:
            choices = [str(c).replace(" ", "").lower() for c in choices]

        # Append per-choice booleans in a fixed order
        for c in choices:
            y_llm1.append(c in llm_answer_1)
            y_llm2.append(c in llm_answer_2)
            y_researcher_1.append(c in researcher_1_ans)
            y_researcher_2.append(c in researcher_2_ans)


def show_pair(name_a, y_a, name_b, y_b):
    k = cohen_kappa_score(y_a, y_b)
    acc = (pd.Series(y_a) == pd.Series(y_b)).mean()
    print(
        f"{name_a} vs {name_b}: kappa={k:.4f}, agreement={acc*100:.2f}% (n={len(y_a)})"
    )


print("\n=== Pairwise agreement (Cohen’s κ and raw agreement) ===")
show_pair(f"{model_1}", y_llm1, "researcher 1", y_researcher_1)
show_pair(f"{model_1}", y_llm1, "researcher 2", y_researcher_2)
show_pair(f"{model_2}", y_llm2, "researcher 1", y_researcher_1)
show_pair(f"{model_2}", y_llm2, "researcher 2", y_researcher_2)
show_pair(f"{model_1}", y_llm1, f"{model_2}", y_llm2)

Calculate precision and recall scores

In [ ]:
import pandas as pd
import os
import json
import math

model = "deepseek"
answer_dir = f"answers/{model}/"

# load data
answers_researcher_1 = pd.read_csv(
    "manual_extraction/answers_researcher_1.csv", sep=";"
)
answers_researcher_2 = pd.read_csv(
    "manual_extraction/answers_researcher_2.csv", sep=";"
)


# normalize the paper key once
def clean_key(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
        .str.replace("_", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace("-", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.lower()
    )


answers_researcher_1["paper_key"] = clean_key(answers_researcher_1["paper"])
answers_researcher_2["paper_key"] = clean_key(answers_researcher_2["paper"])

answers_researcher_1 = answers_researcher_1.sort_values(by="paper_key")
answers_researcher_2 = answers_researcher_2.sort_values(by="paper_key")

tp = fp = fn = 0

for i, paper_fname in enumerate(os.listdir(answer_dir)):
    with open(os.path.join(answer_dir, paper_fname), "r") as f:
        answers = json.load(f)

    # filename-derived key
    paper_clean_name = paper_fname.replace("_", "").replace(" ", "").lower()
    # use prefix match on first 35 chars (your original intent)
    prefix = paper_clean_name[:35]

    # boolean masks — NOTE: .loc with boolean mask (NOT .iloc)
    mask_researcher_1 = answers_researcher_1["paper_key"].str[:35].eq(prefix)
    mask_researcher_2 = answers_researcher_2["paper_key"].str[:35].eq(prefix)

    # optionally enforce uniqueness (skip if no or multi matches)
    if mask_researcher_1.sum() != 1 or mask_researcher_2.sum() != 1:
        print(f"prefix: {prefix}")
        print(f"Researcher 1 paper keys: {answers_researcher_1['paper_key'].str[:35]}")
        print(f"Researcher 2 paper keys: {answers_researcher_2['paper_key'].str[:35]}")
        raise Exception(f"No unique match found for paper {paper_fname}")

    for q in answers["answers"]:

        qname = q["question"]

        # normalize LLM answer to list[str]
        llm_answer = q["answer"]
        if not isinstance(llm_answer, list):
            llm_answer = [llm_answer]
        llm_answer = [str(x).replace(" ", "").lower() for x in llm_answer]

        # fetch human answers as scalars and normalize
        def get_cell(df, mask, col):
            # returns a single normalized string ('' if NaN)
            val = df.loc[mask, col].iloc[0]  # Series -> scalar
            if isinstance(val, float) and math.isnan(val):
                return ""
            return str(val)

        researcher_1_cell = get_cell(answers_researcher_1, mask_researcher_1, qname)
        researcher_2_cell = get_cell(answers_researcher_2, mask_researcher_2, qname)

        researcher_1_answer = (
            researcher_1_cell.replace(" ", "").lower().split(";")
            if researcher_1_cell
            else []
        )
        researcher_2_answer = (
            researcher_2_cell.replace(" ", "").lower().split(";")
            if researcher_2_cell
            else []
        )

        # scoring
        choices = q["choices_ids"]
        for choice in choices:
            c = str(choice).replace(" ", "").lower()
            if c in llm_answer:
                if (c in researcher_2_answer) or (c in researcher_1_answer):
                    tp += 1
                else:
                    fp += 1
            else:
                if (c in researcher_2_answer) and (c in researcher_1_answer):
                    fn += 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")